In [1]:
from owslib.ogcapi.features import Features
import geopandas as gpd
import datetime as dt
from datetime import datetime, timedelta

C:\Users\admin-magstadt\anaconda3\envs\ee\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
OGC_URL = "https://firenrt.delta-backend.com"

w = Features(url=OGC_URL)
w.feature_collections()

['public.eis_fire_lf_newfirepix_nrt',
 'public.eis_fire_snapshot_perimeter_pkey_test',
 'public.eis_fire_snapshot_fireline_nrt',
 'public.eis_fire_lf_fireline_archive',
 'public.eis_fire_lf_perimeter_archive',
 'public.eis_fire_lf_perimeter_nrt',
 'public.eis_fire_lf_nfplist_nrt',
 'public.eis_fire_snapshot_perimeter_nrt',
 'public.eis_fire_lf_nfplist_archive',
 'public.eis_fire_lf_fireline_nrt',
 'public.eis_fire_lf_newfirepix_archive',
 'public.eis_fire_snapshot_newfirepix_nrt',
 'public.eis_public_snapshot_perimeter_testv2',
 'public.eis_fire_fwi',
 'public.st_squaregrid',
 'public.st_hexagongrid',
 'public.st_subdivide']

In [3]:
perm = w.collection("public.eis_fire_snapshot_perimeter_nrt")


In [4]:
perm_q = w.collection_queryables("public.eis_public_snapshot_perimeter_testv2")
perm_q["properties"]

{'geometry': {'$ref': 'https://geojson.org/schema/Geometry.json'},
 'duration': {'name': 'duration', 'type': 'number'},
 'farea': {'name': 'farea', 'type': 'number'},
 'fireid': {'name': 'fireid', 'type': 'number'},
 'flinelen': {'name': 'flinelen', 'type': 'number'},
 'fperim': {'name': 'fperim', 'type': 'number'},
 'geom_counts': {'name': 'geom_counts', 'type': 'number'},
 'isactive': {'name': 'isactive', 'type': 'number'},
 'low_confidence_grouping': {'name': 'low_confidence_grouping',
  'type': 'number'},
 'meanfrp': {'name': 'meanfrp', 'type': 'number'},
 'n_newpixels': {'name': 'n_newpixels', 'type': 'number'},
 'n_pixels': {'name': 'n_pixels', 'type': 'number'},
 'pixden': {'name': 'pixden', 'type': 'number'},
 'primarykey': {'name': 'primarykey', 'type': 'string'},
 'region': {'name': 'region', 'type': 'string'},
 't': {'name': 't', 'type': 'string'}}

In [6]:
## Get the most recent fire perimeters, and 7 days before most recent fire perimeter
most_recent_time = max(*perm["extent"]["temporal"]["interval"])
now = dt.datetime.strptime(most_recent_time, "%Y-%m-%dT%H:%M:%S+00:00")
last_week = now - dt.timedelta(days=1)
last_week = dt.datetime.strftime(last_week, "%Y-%m-%dT%H:%M:%S+00:00")
print("Most Recent Time =", most_recent_time)
print("Last week =", last_week)

Most Recent Time = 2024-04-16T12:00:00+00:00
Last week = 2024-04-15T12:00:00+00:00


In [7]:
fline_q = w.collection_queryables("public.eis_fire_snapshot_fireline_nrt")
fline_collection = w.collection("public.eis_fire_snapshot_fireline_nrt")
fline_q["properties"]

{'wkb_geometry': {'$ref': 'https://geojson.org/schema/Geometry.json'},
 'fireid': {'name': 'fireid', 'type': 'number'},
 'mergeid': {'name': 'mergeid', 'type': 'number'},
 'ogc_fid': {'name': 'ogc_fid', 'type': 'number'},
 't': {'name': 't', 'type': 'string'}}

In [8]:
perm_results = w.collection_items(
    "public.eis_fire_snapshot_perimeter_nrt",
    datetime=most_recent_time,
    limit=1000,
)
perimeters = gpd.GeoDataFrame.from_features(perm_results["features"])

## Get the most recent fire lines
perimeter_ids = perimeters.fireid.unique()
perimeter_ids = ",".join(map(str, perimeter_ids))

fline_results = w.collection_items(
    "public.eis_fire_snapshot_fireline_nrt",
    limit=1000,
    filter="fireid IN ("
    + perimeter_ids
    + ")",  # only the fires from the fire perimeter query above
)
fline = gpd.GeoDataFrame.from_features(fline_results["features"])

In [9]:
perimeters

,geometry,duration,farea,fireid,flinelen,fperim,isactive,meanfrp,n_newpixels,n_pixels,ogc_fid,pixden,t
0,"POLYGON ((-78.02313 25.14461, -78.01385 25.138...",14.0,79.289874,53221,0.0,53.018911,1,0.0,0,540,9,6.810453,2024-04-16T12:00:00
1,"POLYGON ((-77.35377 26.73118, -77.35372 26.731...",18.0,19.476873,50951,0.0,21.971552,1,0.0,0,227,19,11.654848,2024-04-16T12:00:00
2,"POLYGON ((-77.42140 26.72606, -77.42144 26.725...",0.0,0.141000,62853,0.0,1.177624,1,0.0,0,1,20,7.092199,2024-04-16T12:00:00
3,"POLYGON ((-77.10691 26.52785, -77.10691 26.527...",1.0,1.246374,62390,0.0,4.364144,1,0.0,0,5,21,4.011637,2024-04-16T12:00:00
4,"POLYGON ((-81.37111 25.96357, -81.37444 25.968...",0.0,1.100733,62852,0.0,4.302213,1,0.0,0,5,35,4.542429,2024-04-16T12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,"POLYGON ((-106.44779 26.65680, -106.44779 26.6...",3.0,6.499652,60927,0.0,9.990519,1,0.0,0,56,11997,8.615846,2024-04-16T12:00:00
396,"POLYGON ((-107.28732 24.49158, -107.28732 24.4...",0.0,0.141000,63150,0.0,1.177624,1,0.0,0,1,12120,7.092199,2024-04-16T12:00:00
397,"POLYGON ((-110.66052 28.16359, -110.66051 28.1...",0.0,0.141000,63032,0.0,1.177624,1,0.0,0,1,12191,7.092199,2024-04-16T12:00:00
398,"POLYGON ((-109.01684 28.45731, -109.01683 28.4...",0.0,0.141000,63155,0.0,1.177624,1,0.0,0,1,12197,7.092199,2024-04-16T12:00:00


In [10]:
df = gpd.GeoDataFrame.from_features(perm_results["features"])
df

,geometry,duration,farea,fireid,flinelen,fperim,isactive,meanfrp,n_newpixels,n_pixels,ogc_fid,pixden,t
0,"POLYGON ((-78.02313 25.14461, -78.01385 25.138...",14.0,79.289874,53221,0.0,53.018911,1,0.0,0,540,9,6.810453,2024-04-16T12:00:00
1,"POLYGON ((-77.35377 26.73118, -77.35372 26.731...",18.0,19.476873,50951,0.0,21.971552,1,0.0,0,227,19,11.654848,2024-04-16T12:00:00
2,"POLYGON ((-77.42140 26.72606, -77.42144 26.725...",0.0,0.141000,62853,0.0,1.177624,1,0.0,0,1,20,7.092199,2024-04-16T12:00:00
3,"POLYGON ((-77.10691 26.52785, -77.10691 26.527...",1.0,1.246374,62390,0.0,4.364144,1,0.0,0,5,21,4.011637,2024-04-16T12:00:00
4,"POLYGON ((-81.37111 25.96357, -81.37444 25.968...",0.0,1.100733,62852,0.0,4.302213,1,0.0,0,5,35,4.542429,2024-04-16T12:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,"POLYGON ((-106.44779 26.65680, -106.44779 26.6...",3.0,6.499652,60927,0.0,9.990519,1,0.0,0,56,11997,8.615846,2024-04-16T12:00:00
396,"POLYGON ((-107.28732 24.49158, -107.28732 24.4...",0.0,0.141000,63150,0.0,1.177624,1,0.0,0,1,12120,7.092199,2024-04-16T12:00:00
397,"POLYGON ((-110.66052 28.16359, -110.66051 28.1...",0.0,0.141000,63032,0.0,1.177624,1,0.0,0,1,12191,7.092199,2024-04-16T12:00:00
398,"POLYGON ((-109.01684 28.45731, -109.01683 28.4...",0.0,0.141000,63155,0.0,1.177624,1,0.0,0,1,12197,7.092199,2024-04-16T12:00:00


In [11]:
df = df.set_crs("EPSG:4326")
df.explore()

In [12]:
df['centroid'] = df['geometry'].centroid
df

C:\Users\admin-magstadt\AppData\Local\Temp\ipykernel_7676\2837296943.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df['centroid'] = df['geometry'].centroid


,geometry,duration,farea,fireid,flinelen,fperim,isactive,meanfrp,n_newpixels,n_pixels,ogc_fid,pixden,t,centroid
0,"POLYGON ((-78.02313 25.14461, -78.01385 25.138...",14.0,79.289874,53221,0.0,53.018911,1,0.0,0,540,9,6.810453,2024-04-16T12:00:00,POINT (-78.04972 25.10115)
1,"POLYGON ((-77.35377 26.73118, -77.35372 26.731...",18.0,19.476873,50951,0.0,21.971552,1,0.0,0,227,19,11.654848,2024-04-16T12:00:00,POINT (-77.39148 26.71682)
2,"POLYGON ((-77.42140 26.72606, -77.42144 26.725...",0.0,0.141000,62853,0.0,1.177624,1,0.0,0,1,20,7.092199,2024-04-16T12:00:00,POINT (-77.42323 26.72650)
3,"POLYGON ((-77.10691 26.52785, -77.10691 26.527...",1.0,1.246374,62390,0.0,4.364144,1,0.0,0,5,21,4.011637,2024-04-16T12:00:00,POINT (-77.11398 26.52747)
4,"POLYGON ((-81.37111 25.96357, -81.37444 25.968...",0.0,1.100733,62852,0.0,4.302213,1,0.0,0,5,35,4.542429,2024-04-16T12:00:00,POINT (-81.36858 25.96558)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,"POLYGON ((-106.44779 26.65680, -106.44779 26.6...",3.0,6.499652,60927,0.0,9.990519,1,0.0,0,56,11997,8.615846,2024-04-16T12:00:00,POINT (-106.43591 26.65721)
396,"POLYGON ((-107.28732 24.49158, -107.28732 24.4...",0.0,0.141000,63150,0.0,1.177624,1,0.0,0,1,12120,7.092199,2024-04-16T12:00:00,POINT (-107.28914 24.49144)
397,"POLYGON ((-110.66052 28.16359, -110.66051 28.1...",0.0,0.141000,63032,0.0,1.177624,1,0.0,0,1,12191,7.092199,2024-04-16T12:00:00,POINT (-110.66240 28.16338)
398,"POLYGON ((-109.01684 28.45731, -109.01683 28.4...",0.0,0.141000,63155,0.0,1.177624,1,0.0,0,1,12197,7.092199,2024-04-16T12:00:00,POINT (-109.01873 28.45713)


In [27]:
import requests
from datetime import datetime
import smtplib
from email.message import EmailMessage
from datetime import datetime, timedelta
import pandas as pd
# Function to retrieve weather forecast
def get_weather_forecast():
    # Set your API token obtained from the token request page
    api_token = "UNmHisiIutbbSdsvQZGTdZhzzEYiMtuV"

    # Set the base URL for the NOAA API
    #base_url = "https://api.weather.gov/points/40.655253,-105.308434"
    base_url = "https://api.weather.gov/points/40.655253,-105.308434"

    # Set the headers with the API token
    headers = {"token": api_token}

    # Send the HTTP GET request to retrieve the data
    response = requests.get(base_url, headers=headers)

    # Check the response status code
    if response.status_code == 200:
        # Extract the forecast URL from the response
        data = response.json()
        forecast_url = data["properties"]["forecast"]

        # Send a new HTTP GET request to retrieve the forecast data
        forecast_response = requests.get(forecast_url, headers=headers)

        # Check the forecast response status code
        if forecast_response.status_code == 200:
            # Extract the data from the forecast response
            forecast_data = forecast_response.json()
            return forecast_data
        else:
            return None
    else:
        return None

In [ ]:
import csv
from datetime import datetime, timedelta

if __name__ == '__main__':
    # Get today's date
    today = datetime.now().date()

    # Assume get_weather_forecast() is defined elsewhere and retrieves weather forecast data
    forecast_data = get_weather_forecast()

    if forecast_data:
        # Define periods for forecast (including the next five days)
        periods = [
            {"label": "Today", "index": 0},
            {"label": "Tonight", "index": 1},
        ]
        
        # Add the next five days to the periods list (only considering day forecasts)
        for i in range(2, 14, 2):
            periods.append({"label": (today + timedelta(days=i//2)).strftime("%A"), "index": i})
        
        # Initialize an empty list to store weather data rows
        weather_data = [today.strftime("%Y-%m-%d")]
        
        # Prepare weather information and append to weather_data list
        for period in periods:
            index = period["index"]
            detailed_forecast = forecast_data["properties"]["periods"][index]["detailedForecast"]
            weather_data.append(detailed_forecast)
        
        # Read the existing CSV and append the new row
        csv_file_path = "C:\\Users\\admin-magstadt\\Desktop\\WeatherArchive\\WeatherArchive_BellvueCO.csv"
        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(weather_data)
        
        print("Weather forecast data appended to CSV successfully!")
    else:
        print("Failed to retrieve weather forecast.")
